# feature engineering

In order to for the model to learn better, we will create some new features from the existing data. This is called feature engineering.

First, I think it is a good idea to reduce the number of samples in the dataset. The current dataset has 28 million samples. Before we train the model on the entire dataset, we will create a smaller dataset with 100K samples. This will help us to iterate faster and test our feature engineering ideas.